In [1]:
typealias Amplitude Complex{Float64}
typealias CV Array{Amplitude,1}
typealias CT Tuple{Amplitude}
typealias CM Array{Amplitude,2}

using Base.Test

type Qreg
    state::CV
end

Qreg(nbits::Int) = begin
    result = Qreg(zeros(1 << nbits))
    # We set the first index to 1 because initially the register MUST be in a particular state, 000...
    result.state[1] = 1
    result
end

is_bin_string(s::String) = reduce(&, [c == '0' || c == '1' for c in s])

"""
Convert the passed string in "[0|1]+" format to a binary number
"""
bin_str_to_int(s::String) = begin
    @assert is_bin_string(s)
    sr = reverse(s)
    reduce(|, [(((sr[i]=='1') & 1) << (i-1)) for i = 1:length(sr)])
end


"""
Create a quantum register from a string of the form |[0,1]+>
"""
Qreg(s::String) = begin
    @assert s[1] == '|'
    @assert s[end] == '>'
    vals = s[2:end-1]
    result = Qreg(zeros(1 << length(vals)))
    result.state[bin_str_to_int(vals)+1] = 1
    result
end


Base.copy(r::Qreg) = begin
    result = Qreg(length(r.state))
    result.state = copy(r.state)
    result
end

Base.:≈(lhs::Qreg, rhs::Qreg) = lhs.state ≈ rhs.state


"""
Creates a "cat" state such that all bits are in a 50% superposition
of 0 and 1
"""
cat(nbits::Int) = begin
    result = Qreg(nbits)
    for i=1:nbits
        result=apply(H,result,i)
    end
    result
end


probs(r::Qreg) = begin
    num_digits = Int(log2(length(r.state)))
    reps = [ @sprintf("%s: %2.2f", bits(s-1)[end-num_digits:end], abs(r.state[s])^2) for s=1:length(r.state)]
    print(join(reps, "\n"),"\n")
end

# we shouldn't have to make our own copy constructor surely?
@testset "qreg" begin
    @test is_bin_string("001110") == true
    @test is_bin_string("00111f") == false
    @test bin_str_to_int("0") == 0
    @test bin_str_to_int("010") == 2
    @test Qreg(1) ≈ Qreg([1+0im, 0+0im])
    @test Qreg("|0>") ≈ Qreg([1+0im, 0+0im])
    @test Qreg("|1>") ≈ Qreg([0+0im, 1+0im])
end

probs(Qreg("|0>"))

Test Summary: | Pass  Total
  qreg        |    7      7
00: 1.00
01: 0.00


In [2]:
?@sprintf

```
@sprintf("%Fmt", args...)
```

Return `@printf` formatted output as string.

```
julia> s = @sprintf "this is a %s %15.1f" "test" 34.567;

julia> println(s)
this is a test            34.6
```


In [3]:
abstract BasicGate

abstract MatrixGate <: BasicGate

state_pairs(n_states::Int, bit_index::Int) = begin
    ups = [i for i in 1:n_states if ((i-1)>>(bit_index-1)) & 1 == 0]
    downs = ups + 1 << (bit_index-1)
    zip(ups, downs)
end
# gets the indices of the states in a register corresponding to a bit number
state_pairs(r::Qreg, bit_index::Int) = state_pairs(length(r.state), bit_index)

Base.:|(g::MatrixGate, r::Qreg, i::Int) = begin
end


In [4]:
state_pairs(Qreg(2),2)

Base.Zip2{Array{Int64,1},Array{Int64,1}}([1,2],[3,4])

In [5]:


r = Qreg(3)
state_pairs(r,1)
@testset "indices test" begin
    @test state_pairs(r,1) == [(1,2), (3,4), (5,6), (7,8)]
    @test state_pairs(r,2) == [(1,3),(2,4),(5,7),(6,8)]
    @test state_pairs(r,3) == [(1,5), (2,6), (3,7), (4,8)]
end


indices test: Test Failed
  Expression: state_pairs(r,1) == [(1,2),(3,4),(5,6),(7,8)]
   Evaluated: Base.Zip2{Array{Int64,1},Array{Int64,1}}([1,3,5,7],[2,4,6,8]) == Tuple{Int64,Int64}[(1,2),(3,4),(5,6),(7,8)]
 in record(::Base.Test.DefaultTestSet, ::Base.Test.Fail) at .\test.jl:428
 in do_test(::Base.Test.Returned, ::Expr) at .\test.jl:281
 in macro expansion; at .\In[5]:6 [inlined]
 in macro expansion; at .\test.jl:672 [inlined]
 in anonymous at .\<missing>:?
 in include_string(::String, ::String) at .\loading.jl:441
 in execute_request(::ZMQ.Socket, ::IJulia.Msg) at C:\Users\Victor.Putz\.julia\v0.5\IJulia\src\execute_request.jl:156
 in eventloop(::ZMQ.Socket) at C:\Users\Victor.Putz\.julia\v0.5\IJulia\src\eventloop.jl:8
 in (::IJulia.##13#19)() at .\task.jl:360
indices test: Test Failed
  Expression: state_pairs(r,2) == [(1,3),(2,4),(5,7),(6,8)]
   Evaluated: Base.Zip2{Array{Int64,1},Array{Int64,1}}([1,2,5,6],[3,4,7,8]) == Tuple{Int64,Int64}[(1,3),(2,4),(5,7),(6,8)]
 in record(::Base

LoadError: Some tests did not pass: 0 passed, 3 failed, 0 errored, 0 broken.

-1.0 - 1.2246467991473532e-16im

In [41]:
abstract BasicGate

abstract MatrixGate <: BasicGate

type BitwiseMatrixGate <: MatrixGate
    m::CM
end

X = BitwiseMatrixGate([0 1; 1 0])
H = BitwiseMatrixGate(1/√2 * [1 1; 1 -1])
Z = BitwiseMatrixGate([1 0; 0 -1])
R(θ) = BitwiseMatrixGate([exp(-0.5*θ*im) 0; 0 (exp(0.5*θ*im))])

apply(g::BitwiseMatrixGate, r::Qreg, qbit::Int) = begin
    result = copy(r)
    pairs = state_pairs(r, qbit)
    for p in pairs
        pi = collect(p)
        result.state[pi] = g.m*result.state[pi]
    end
    result
end

type ControlledMatrixGate <: MatrixGate
    g::BitwiseMatrixGate
    num_control_bits::Int
end

"""
Returns the state pairs affecting the bit `qbit` where the control bits `control_bits` are '
"""
controlled_pairs(n_states::Int, control_bits::Array{Int}, qbit::Int) = begin
    mask = mask_for_bits(control_bits)
    # now get all pairs which affect the qbit
    pairs = state_pairs(n_states, qbit)
    # but only keep the ones where the control registers are 1
    result = [p for p in pairs if ((p[1]-1) & mask) == mask]
end

controlled_pairs(r::Qreg, control_bits::Array{Int}, qbit::Int) = controlled_pairs(length(r.state), control_bits, qbit)

apply(g::ControlledMatrixGate, r::Qreg, control_bits::Array{Int}, qbit::Int) = begin
    @assert length(control_bits) == g.num_control_bits
    result = copy(r)
    for p in controlled_pairs(r, control_bits, qbit)
        pi = collect(p)
        result.state[pi] = g.g.m*result.state[pi]
    end
    result
end

CNOT = ControlledMatrixGate(X, 1)

@testset "Basic Gates" begin
    @test controlled_pairs(8, [2], 1) == [(3,4), (7,8)]
    
    @test apply(X, Qreg("|0>"), 1) ≈ Qreg("|1>")
    @test apply(X, Qreg("|0100>"), 3) ≈ Qreg("|0000>")
    @test apply(CNOT, Qreg("|10>"),[2],1) ≈ Qreg("|11>")
    @test apply(CNOT, Qreg("|00>"),[2],1) ≈ Qreg("|00>")
end



Test Summary: | Pass  Total
  Basic Gates |    5      5


Base.Test.DefaultTestSet("Basic Gates",Any[Test Passed
  Expression: controlled_pairs(8,[2],1) == [(3,4),(7,8)]
   Evaluated: Tuple{Int64,Int64}[(3,4),(7,8)] == Tuple{Int64,Int64}[(3,4),(7,8)],Test Passed
  Expression: apply(X,Qreg("|0>"),1) ≈ Qreg("|1>")
   Evaluated: Qreg(Complex{Float64}[0.0+0.0im,1.0+0.0im]) isapprox Qreg(Complex{Float64}[0.0+0.0im,1.0+0.0im]),Test Passed
  Expression: apply(X,Qreg("|0100>"),3) ≈ Qreg("|0000>")
   Evaluated: Qreg(Complex{Float64}[1.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im]) isapprox Qreg(Complex{Float64}[1.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im,0.0+0.0im]),Test Passed
  Expression: apply(CNOT,Qreg("|10>"),[2],1) ≈ Qreg("|11>")
   Evaluated: Qreg(Complex{Float64}[0.0+0.0im,0.0+0.0im,0.0+0.0im,1.0+0.0im]) isapprox Qreg(Complex{F

Int64) in module Main at In[24]:38 overwritten at In[41]:40.


In [7]:

"""
Picks a state as if a random number generator had come up with p=with_probability
"""
pick_state(r::Qreg, with_probability::Float64) = begin
    accum = 0
    index = 1
    while accum < with_probability && index < length(r.state)
        accum += abs(r.state[index])^2
        index += 1
    end
    index -= 1
    index
end

@test pick_state(apply(X, Qreg(3), 3), 0.5) == 5


Test Passed
  Expression: pick_state(apply(X,Qreg(3),3),0.5) == 5
   Evaluated: 5 == 5

In [25]:
controlled_pairs(4, [1], 2)

1-element Array{Tuple{Int64,Int64},1}:
 (2,4)

In [9]:
"""
The value of bits `bits` if the state `state` is picked.
Uses 1-based indexing!  This may not be desired.  Think on this
(ie state 1 = |0>, state 2 = |1>, etc)
"""
bits_for_state(state::Int, bits::Array{Int}) = [((((state - 1) >> (b-1)) & 1) == 1) for b in bits]

"""
The "mask" for a set of bit indices--basically the bits corresponding to the 
appropriate indices are set (ie `mask_for_bits([1, 3])==5`)
"""
mask_for_bits(bits) = reduce(|, [1<<(p-1) for p in bits])

"""
The "value" for a set of bit indices given a state
"""
state_val(res::Array{Bool}, bits::Array{Int}) = reduce(|, [((res[i] & 1) << (bits[i]-1)) for i =1:length(bits)])

"""
Measures the register at the picked state given.

Marks the register so that a picked state with a particular set of bits eliminates 
disagreeing states (ie if bit 2 is measured to be 1, all states with bit 2 = 0 
must be set to zero probability)

Returns the array of bit values, and the new Qreg (after measurement)
"""
measure_picked_state(r::Qreg, pick::Int, bits::Array{Int}) = begin
    # res: basically, which bits are true vs false in that picked state--ALL bits
    result = bits_for_state(pick, bits)
    # mask: which bits are *involved* in this measurement
    mask = mask_for_bits(bits)
    # val: values of the set bits in the given mask
    val = state_val(result, bits)
    #display(result)
    #display(mask)
    #display(val)
    
    zero_states = [i for i in 1:length(r.state) if (mask & (i-1)) != val]
    #display(zero_states)
    keep_states = [i for i in 1:length(r.state) if (mask & (i-1)) == val]
    #display(keep_states)
    
    rr = copy(r)
    rr.state[zero_states] = 0
    renorm = sum(abs(rr.state[keep_states]) .^ 2)
    #display(renorm)
    rr.state[keep_states] *= 1/√renorm
    
    (result, rr)
end

@testset "Measuring tests" begin
    @test bits_for_state(1, [1 2 3]) == [false false false]
    @test bits_for_state(2, [1 2]) == [true false]
    @test bits_for_state(7, [1 2 3]) == [false true true]
    @test mask_for_bits([1 3]) == 5
    
    @test measure_picked_state(apply(H,Qreg(1),1),1,[1])[1] == [false]
    @test measure_picked_state(apply(H,Qreg(1),1),1,[1])[2] ≈ Qreg(1)
end


Test Summary:   | Pass  Total
  Measuring tests |    6      6


In [10]:
r = cat(3)
val, r2 = measure_picked_state(r, 3, [2])
display(val)
probs(r2)

1-element Array{Bool,1}:
 true

0000: 0.00
0001: 0.00
0010: 0.25
0011: 0.25
0100: 0.00
0101: 0.00
0110: 0.25
0111: 0.25


In [31]:
bell_pair(r, b1, b2) = begin
    r2 = apply(H, r, b1)
    r3 = apply(CNOT, r2, [b1], b2)
    r3
end
probs(bell_pair(Qreg("|00>"), 2, 1))

000: 0.50
001: 0.00
010: 0.00
011: 0.50


In [33]:
?with

search: withenv with_rounding with_bigfloat_precision endswith strwidth

Couldn't find with
Perhaps you meant wait, write, which, bits, coth, sinh, withenv, widen or while


No documentation found.

Binding `with` does not exist.


In [34]:
using ParserCombinator


INFO: Precompiling module AutoHashEquals.


In [39]:
q_const = E"|" + Pattern(r"[01]+") + E">"

parse_one("|001>", q_const )

1-element Array{Any,1}:
 "001"